In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

profesores = []

for pagina in range(2): 
    url = f"https://cris.utec.edu.pe/es/persons/?page={pagina}" if pagina > 0 else "https://cris.utec.edu.pe/es/persons/"
    print(f"🌐 Visitando página {pagina + 1}: {url}")
    driver.get(url)

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "li.grid-result-item")))
    time.sleep(2)  

    soup = BeautifulSoup(driver.page_source, "html.parser")

    for i, item in enumerate(soup.select("li.grid-result-item")):
        container = item.select_one("div.result-container")
        if not container:
            continue

        nombre_tag = container.select_one("h3.title span")
        nombre = nombre_tag.get_text(strip=True) if nombre_tag else None

        correo_tag = container.select_one("ul.relations.email li.email a")
        correo = correo_tag.get_text(strip=True) if correo_tag else None

        detalles = []
        for li in container.select("ul.relations.organisations li"):
            span_text = li.get_text(separator=" ", strip=True)
            detalles.append(span_text)

        while len(detalles) < 4:
            detalles.append("")

        print(f"✅ Página {pagina+1} - [{i}] {nombre} | {correo} | {detalles[0]}")

        profesores.append({
            "nombre": nombre,
            "correo": correo,
            "afiliacion_1": detalles[0],
            "afiliacion_2": detalles[1],
            "afiliacion_3": detalles[2],
            "afiliacion_4": detalles[3]
        })

driver.quit()

df = pd.DataFrame(profesores)
df.to_csv("profesores_utec.csv", index=False, encoding="utf-8-sig")
print(f"✅ CSV guardado con {len(df)} profesores")

🌐 Visitando página 1: https://cris.utec.edu.pe/es/persons/
✅ Página 1 - [0] José Manuel Aguilar Alfaro | jaguilar@utec.edu.pe | Grupo de Investigación en Analítica de Datos y Ciencias de Decisión (DADS) - Investigador
✅ Página 1 - [1] Grace Alexandrino Ocaña | None | Departamento de Humanidades, Artes y Ciencias Sociales - Profesor Auxiliar
✅ Página 1 - [2] Helard Alvarez Sanchez | halvarez@utec.edu.pe | Departamento de Ingeniería Mecánica y de la Energía
✅ Página 1 - [3] Claudia Antonini Bova | cantonini@utec.edu.pe | Departamento de Ingeniería Industrial y Gestión de la Ingeniería
✅ Página 1 - [4] Sergio Aranda Egusquiza | saranda@utec.edu.pe | Departamento de Ingeniería Electrónica y Mecatrónica
✅ Página 1 - [5] Patricia Araujo Pantoja | paraujo@utec.edu.pe | Grupo de investigación en energía y minas - Investigador
✅ Página 1 - [6] Andrés Julián Arias Moreno | aariasm@utec.edu.pe | Departamento de Bioingeniería e Ingeniería Química
✅ Página 1 - [7] Luis Bedriñana Mera | lbedrinana@u